### Importamos las librerías

In [2]:
!pip install transformers[torch] torch streamlit datasets scikit-learn matplotlib


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
#importamos las bibliotecas necesarias
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preparar datos de ejemplo

In [2]:
#Paso 1: Definir los datos de entrenamiento
texts = [
     "Los paneles solares son una fuente eficiente de energía renovable.",
     "El carbón sigue siendo una fuente importante de energía en muchos países.",
     "La energía eólica está ganando popularidad en todo el mundo.",
     "Las centrales nucleares son controversiales pero producen energía sin emisiones de CO2.",
     "La biomasa es una forma de energía renovable que utiliza materiales orgánicos.",
 ]
labels = [1, 0, 1, 0, 1]  # 1 para energía renovable, 0 para no renovable

### Crear un dataset de Hugging Face

In [3]:
#paso 2: Crear un dataset de Hugging face
#Convierte los datos en un objeto DatasetDict para poder manupularlo fácilmente en el entrenamiento
dataset = Dataset.from_dict({"text": texts, "label": labels})

### Cargar el tokenizador y el modelo

In [4]:
 # paso 3: Cargar el tokenizador y el modelo preentrenado BERT
 # Bert-base-uncased es un modelo de BERT en ingles sin distinción entre mayúsculas y minusculas
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 ### Función de preprocesamiento

In [5]:
#Paso 4: Función de preprocesamiento: Función para convertir los textos en tokens
def tokenize_function(examples):
     return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 5/5 [00:00<00:00, 345.32 examples/s]


### Dividir el dataset

In [6]:
 # Paso 5: Dividir el dataset
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

### Función de cálculo de métricas

In [7]:
# Paso 6: Función de cálculo de métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

### Configurar el entrenamiento

In [8]:
# Paso 7: Configurar el entrenamiento
training_args = TrainingArguments(
     output_dir="./results",
     num_train_epochs=3,
     per_device_train_batch_size=16,
     per_device_eval_batch_size=64,
     warmup_steps=500,
     weight_decay=0.01,
     logging_dir='./logs',
     logging_steps=10,
     evaluation_strategy="epoch",
 )

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Crear el Trainer

In [9]:
# Paso 8: Crear el Trainer
trainer = Trainer(
     model=model,
     args=training_args,
     train_dataset=tokenized_dataset["train"],
     eval_dataset=tokenized_dataset["test"],
     compute_metrics=compute_metrics,
 )

### Entrenar el modelo

In [10]:
# Paso 9: Entrenar el modelo
trainer.train()

: 


### Evaluar el modelo

In [ ]:
# Paso 10: Evaluar el modelo
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 1.1034537553787231, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.7599, 'eval_samples_per_second': 1.316, 'eval_steps_per_second': 1.316, 'epoch': 3.0}


c:\BootcampIA\bootcamp\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Hacer una predicción

In [ ]:
# Paso 11: Hacer una predicción
text = "La energía geotérmica aprovecha el calor de la Tierra."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
prediction = torch.argmax(outputs.logits).item()
print(f"Predicción: {'Energía Renovable' if prediction == 1 else 'Energía No Renovable'}")

Predicción: Energía Renovable


In [ ]:
jupyter